Turns reservation raw and resources raw data into csv format

In [8]:
# Install required packages
# %pip install psycopg2
# %pip install pandas
# %pip install python-dotenv

In [9]:
import pandas as pd
import psycopg2.pool as pg_pool
import dotenv

dotenv.load_dotenv()

HOSTNAME = dotenv.get_key(".env", "HOSTNAME")
USERNAME = dotenv.get_key(".env", "USERNAME")
PASSWORD = dotenv.get_key(".env", "PASSWORD")
PORT = dotenv.get_key(".env", "PORT")
DATABASE = dotenv.get_key(".env", "DATABASE")

pool = pg_pool.SimpleConnectionPool(
    1,
    10,
    user=USERNAME,
    password=PASSWORD,
    host=HOSTNAME,
    port=PORT,
    database=DATABASE
)

In [10]:
conn = pool.getconn()
#spaces_df = pd.read_sql('SELECT * FROM cds_classroom_analysis.mv_spaces_raw LIMIT 100;', conn)
res_df = pd.read_sql('SELECT * FROM cds_classroom_analysis.mv_reservations_raw', conn)
resources_df = pd.read_sql('SELECT * FROM cds_classroom_analysis.mv_resources_raw', conn)
#spaces_df.head()

C:\Users\brent\AppData\Local\Temp\ipykernel_38832\3611130638.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  res_df = pd.read_sql('SELECT * FROM cds_classroom_analysis.mv_reservations_raw', conn)
C:\Users\brent\AppData\Local\Temp\ipykernel_38832\3611130638.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resources_df = pd.read_sql('SELECT * FROM cds_classroom_analysis.mv_resources_raw', conn)


In [11]:
#org_df=pd.read_sql('SELECT * FROM cds_classroom_analysis.mv_organizations_raw LIMIT 100', conn)

In [12]:
import json

def json_column_expand(df, col = 'data'):
    try:
        # Use the correct dataframe name: spaces_df
        json_records = df[col].tolist()
    except KeyError:
        print("Error: 'json_data' column not found. Please verify the JSON column name.")
        exit()
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON at index {e.doc[e.pos-10:e.pos+10]}: {e}")
        exit()

    # 2.2 Use json_normalize to flatten the list of dictionaries into a new DataFrame
    # The 'sep' argument helps distinguish nested columns (e.g., config_size)
    df_tabular = pd.json_normalize(
        json_records,
        sep='_'
    )

    # --- 3. Merging (Optional): Combine with Non-JSON Columns ---

    # Identify and drop the original JSON column from the raw dataframe (spaces_df)
    df_non_json = df.drop(columns=[col])

    # Concatenate the non-JSON columns with the new tabular data
    df_final = pd.concat([df_non_json, df_tabular], axis=1)
    return df_final

In [13]:
res_no_json = json_column_expand(res_df, 'data')
print(res_no_json.columns)
res_no_json.head()

Index(['id', 'dl_inserted_at', 'reservations_engine',
       'reservations_reservation_event_id',
       'reservations_reservation_event_name',
       'reservations_reservation_profile_id',
       'reservations_reservation_event_title',
       'reservations_reservation_last_mod_dt',
       'reservations_reservation_event_end_dt',
       'reservations_reservation_pre_event_dt',
       'reservations_reservation_profile_name',
       'reservations_reservation_event_locator',
       'reservations_reservation_last_mod_user',
       'reservations_reservation_post_event_dt',
       'reservations_reservation_event_start_dt',
       'reservations_reservation_expected_count',
       'reservations_reservation_reservation_id',
       'reservations_reservation_event_type_name',
       'reservations_reservation_organization_id',
       'reservations_reservation_event_type_class',
       'reservations_reservation_registered_count',
       'reservations_reservation_registration_url',
       'reservati

,id,dl_inserted_at,reservations_engine,reservations_reservation_event_id,reservations_reservation_event_name,reservations_reservation_profile_id,reservations_reservation_event_title,reservations_reservation_last_mod_dt,reservations_reservation_event_end_dt,reservations_reservation_pre_event_dt,...,reservations_reservation_space_reservation_default_layout_capacity,reservations_reservation_space_reservation_selected_layout_capacity,reservations_reservation_space_reservation,reservations_reservation_resource_reservation_resource_id,reservations_reservation_resource_reservation_resource_name,reservations_reservation_resource_reservation_resource_count,reservations_reservation_resource_reservation_resource_status,reservations_reservation_resource_reservation_resource_instructions,reservations_reservation_resource_reservation_resource_instruction_id,reservations_reservation_resource_reservation
0,32360540,2025-10-08 18:10:21.664261,accl,600753,IN PROGRESS: Sukkah Reservation,743351,,2025-03-10T09:15:07-04:00,2025-10-15T19:00:00-04:00,2025-10-06T18:00:00-04:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32522597,2025-10-08 18:10:21.846849,accl,603826,Sukkah,746948,,2025-03-10T09:19:13-04:00,2025-10-08T22:00:00-04:00,2025-10-08T07:00:00-04:00,...,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31096525,2025-10-08 18:10:22.020089,accl,556897,Photonics Center Event,690107,,2024-09-06T16:57:14-04:00,2025-10-08T19:00:00-04:00,2025-10-08T07:00:00-04:00,...,180.0,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31537791,2025-10-08 18:10:22.190143,accl,573769,Head and Neck Symposium- Setup,711963,,2025-01-13T13:05:40-05:00,2025-10-08T18:00:00-04:00,2025-10-08T08:00:00-04:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34093926,2025-10-08 18:10:22.360661,accl,630635,MJE Conference,779057,MJE Conference,2025-09-30T15:07:34-04:00,2025-10-08T19:30:00-04:00,2025-10-08T08:00:00-04:00,...,NaN,NaN,"[{'shared': 'F', 'space_id': 5848, 'layout_id'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
res_no_json.to_csv('reservation_data.csv', index = False)

In [18]:
resources_no_json = json_column_expand(resources_df, 'data')
print(resources_no_json.columns)
resources_no_json.to_csv('resources_data.csv', index = False)

Index(['id', 'dl_inserted_at', 'resources_engine', 'resources_resource_crc',
       'resources_resource_stock_crc', 'resources_resource_stock_status',
       'resources_resource_stock_stock_level',
       'resources_resource_stock_version_nbr',
       'resources_resource_stock_stock_end_dt',
       'resources_resource_stock_stock_start_dt', 'resources_resource_status',
       'resources_resource_category_crc', 'resources_resource_category_status',
       'resources_resource_category_category_id',
       'resources_resource_category_category_name',
       'resources_resource_category_category_defn_state',
       'resources_resource_comments', 'resources_resource_favorite',
       'resources_resource_edit_perm', 'resources_resource_comment_id',
       'resources_resource_assign_perm', 'resources_resource_last_mod_dt',
       'resources_resource_resource_id', 'resources_resource_instructions',
       'resources_resource_last_mod_user', 'resources_resource_resource_name',
       'resources